### **Tasks Review and Procedure Creation**

In this exercise you must create a `pool of connection`s and` get a connection from the pool` to implement the stored procedures to complete the following tasks:
* Close the pool after !

#### `Tasks`
* Establish a connection by importing `MySQLConnectionPool` and creating a pool with two connections.

* Create and call a stored procedure named `PeakHour`s that identifies the peak, or busiest hour, for the restaurant based on the number of bookings.

* Create and call a stored procedure named `GuestStatus` that outputs status of each guest’s order based on which employee is assigned to the order.

`Stored Procedures`
Stored procedures are created to carry out routine operations on MySQL databases. They are consistent and make sure that the written SQL queries in the procedures are executed in the same way every time you call the stored procedure.

A stored procedure is created only once, and you store it in the MySQL database. You can call the stored procedures as many times as you need in your Python-based application.

In [2]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

db_config = {
    'database': 'little_lemon_db',
    'user': 'root',
    'password': ''
}

try:
    pool_a = MySQLConnectionPool(pool_name = "pool_a",
                           pool_size = 2, #default is 5
                           **db_config) # unpack argument in dictionary
    
    print("The connection pool is created with a name: ",pool_a.pool_name)
    print("The pool size is:",pool_a.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)


# Get connection & Create Cursor
procedure_connection = pool_a.get_connection()
procedure_cursor = procedure_connection.cursor()

The connection pool is created with a name:  pool_a
The pool size is: 2


### **Task 2: Implement `Peak Hours` Procedure**

In [4]:
peak_hours_query = """
CREATE PROCEDURE PeakHours()
BEGIN
    SELECT
        HOUR(BookingSlot) AS Booking_Hour,
        COUNT(*) AS bookings_for_hour
    FROM Bookings
    GROUP BY Booking_Hour
    ORDER BY bookings_for_hour DESC;
END
"""
# run query to create procuedure
procedure_cursor.execute(peak_hours_query)
# Call procedure
procedure_cursor.callproc('PeakHours')
# stores results in a variable called dataset
dataset = next(procedure_cursor.stored_results())
# column_names
print(dataset.column_names)
# Print sorted data with for loop
for row in dataset.fetchall():
    print(row)

('Booking_Hour', 'bookings_for_hour')
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


### **Task 3: Implement a stored procedure `GuestStatus`**

In [6]:
guest_status_query = """
CREATE PROCEDURE GuestStatus()
BEGIN
    SELECT
        CONCAT(B.GuestFirstName, ' ', B.GuestLastName) AS guest_name,
        CASE
            WHEN E.Role in ('Manager', 'Assistant Manager') THEN 'Ready to pay'
            WHEN E.Role = 'Head Chef' THEN 'Ready to serve'
            WHEN E.Role = 'Assistant Chef' THEN 'Preparing Order'
            WHEN E.Role = 'Head Waiter' THEN 'Order served'
            ELSE 'No Status'
        END AS guest_order_status
    FROM Bookings AS B
    LEFT JOIN Employees AS E
    USING(EmployeeID);
END
"""
# run query to create procuedure
procedure_cursor.execute(guest_status_query)
# Call procedure
procedure_cursor.callproc('GuestStatus')
# stores results in a variable called dataset
dataset = next(procedure_cursor.stored_results())
# column_names
print(dataset.column_names)
# Print sorted data with for loop
for row in dataset.fetchall():
    print(row)

('guest_name', 'guest_order_status')
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')


In [7]:
print("\nReturning the connection back to the pool.")
procedure_connection.close()
print("The connection is placed back into the pool for the next user to connect.")


Returning the connection back to the pool.
The connection is placed back into the pool for the next user to connect.
